# RAG

This work will look at the implementation of RAG within NHS England. This notebook contains a simple RAG pipeline which can work with both RAG turned on, and RAG turned off (relying only on the models innate "knowledge"). 

## Setup

In [5]:
import glob
import os

import toml
from dotenv import load_dotenv

import src.models as models

from tqdm import tqdm

config = toml.load("config.toml")
load_dotenv(".secrets")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("anthropic_key")

if config['DEV_MODE']:
    config['PERSIST_DIRECTORY'] += "/dev"


First we initialise the RAG pipeline - this is an object which links the vector-store, and the LLM, so when you pass a query in it get passed back into the database, and then returns the response.

There are also methods for adding documents to the database.

In [6]:
rag_pipeline = models.RagPipeline(config['EMBEDDING_MODEL'], config['PERSIST_DIRECTORY'])

c:\Users\Will Poulett\OneDrive - NHS\Documents\RAG\ds_251_RAG\.venv\Lib\site-packages\langchain_community\llms\anthropic.py:180: UserWarning: This Anthropic LLM is deprecated. Please use `from langchain_community.chat_models import ChatAnthropic` instead
  warnings.warn(


need to fill the database if it's empty (this might take 5 mins or so the first time, unless you've got a nice graphics card!)

In [3]:
# Add documents if there are non - if in DEV mode, don't add any more (if it's not empty)
if len(rag_pipeline.vectorstore.get()['documents']) == 0 or (not config['DEV_MODE']):
    rag_pipeline.load_documents()  

## RAG vs Non-RAG

In [4]:
question = "Explain the main benefits of Reproducible Analytical Pipelines (RAP)"

result = rag_pipeline.answer_question(question, rag=False)

print(result)

c:\Users\Will Poulett\OneDrive - NHS\Documents\RAG\ds_251_RAG\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



The main benefits of Reproducible Analytical Pipelines (RAP) are:

1. Improved replicability: Using RAPs helps ensure your analysis can be replicated by colleagues or your future self, improving scientific rigor. Automating data processing and analysis steps makes it easier to reproduce results.

2. Reuse and sharing: RAPs make analysis code and workflows reusable by your future self or others in your organization. Packaging code and data together facilitates collaboration and extensions of your work.

3. Efficiency gains: With automated steps for data cleaning, processing, and analysis, RAPs save time compared to manual work. Changes to the analysis flow can be quickly propagated.

4. Better documentation: RAPs explicitly lay out analysis steps and the code underlying them. This helps serve as documentation, refreshing your memory or informing new team members.

5. Error reduction: Automating analysis steps leaves less room for manual error during data manipulation. Automated trackin

Now we will run with  **RAG** turned on. You'll see it spits out a bunch of stuff, as it was set to be verbose - namely, it gives back the completed prompt it submitted to the LLM, followed by the answer - you can see the chunks of documents it found.

In [5]:
result = rag_pipeline.answer_question(question, rag=True)

print(result)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that helps people with their questions. You are not a replacement for human judgement, but you can help humansmake more informed decisions. If you are asked a question you cannot answer based on your following instructions, you should say so.Be concise and professional in your responses.

 Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). If you don't know the answer, just say that you don't know. Don't try to make up an answer. ALWAYS return a "SOURCES" part in your answer.

Example 1: "**RAP** is to be the foundation of analyst training. SOURCES: (goldacre_review.txt)"
Example 2: "Open source code is a good idea because:
* it's cheap (goldacre_review.txt)
* it's easy for people to access and use (open_source_guidlines.txt)
* it's easy to share (goldacre_review.txt)

SOURCES: (golda

d:\PycharmProjects\ds_251_RAG\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

> Finished chain.

The main benefits of Reproducible Analytical Pipelines (RAP) are:

1. They deliver work more efficiently and reliably by minimizing manual steps, which speeds up analysis, allows easy re-running of the analysis pipeline, and makes it easier to share code for collaboration

2. They use open source software like R and Python that guarantees all users have access to the tools, and emphasizes documentation, flexibility, and extensibility

3. They produce analyses and code that are reusable, auditable, efficient, high quality, and more likely to be free of errors

4. They support open sharing of code and analyses, which allows wider review to identify problems and offer improvements, drives re-use and capacity building across the system, and builds trust in the analyses and statistics produced

SOURCES: (goldacre_review.txt)
